# merging DSS_SINIESTROS_AUTOS + (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# general configs
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Merging DSS_SINIESTROS_AUTOS + merged (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

## Loading DSS_SINIESTROS_AUTOS

In [3]:
# path to dataset
path_sini = "../../../data/interim/4. transformed/DSS_SINI_AUTOS_ID-red_col-newcols-dropDuplic-NUM_SECU_EXPED_UNIC.feather"

if not(os.path.exists(path_sini) and os.path.isfile(path_sini)):
    raise Exception("File {} doesn't exists.".format(path_sini))

In [4]:
df_sini = pd.read_feather(path_sini)
df_sini.info()

/home/voolkia/anaconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144995 entries, 0 to 1144994
Data columns (total 44 columns):
CATASTROFICO              1144989 non-null object
CERRADURA_BAUL            504402 non-null object
CERRADURA_DERECHA         504402 non-null object
CERRADURA_IZQUIERDA       504402 non-null object
CODIGO_BAJA               1562 non-null float64
CODIGO_CARATULA           30240 non-null float64
CODIGO_REAPERTURA         58512 non-null float64
CODIGO_VEHICULO           1144905 non-null float64
COD_ACT_BENEF             463893 non-null float64
COD_CAUSA_SINI            1144995 non-null float64
COD_POST_OCURRENCIA       1144991 non-null float64
COD_POST_POLIZA           1144971 non-null float64
COD_POST_TERC             194056 non-null float64
COD_RAMO                  1144995 non-null int64
COD_RIES                  1144995 non-null float64
CONDICION_ROBO_EXP50      13085 non-null float64
DANOS_MATERIALES          1144989 non-null object
DESCRIPCION_TIPO          1144993 non-nul

<b>DSS_SINIESTROS_AUTOS: has both keys NUM_SECU_EXPED & NUM_SECU_POL</b>

In [5]:
# checking ID is unique
df_sini["NUM_SECU_EXPED"].nunique() / len(df_sini)

1.0

In [6]:
# checking ID is unique
df_sini["NUM_SECU_POL"].nunique() / len(df_sini)

0.6410001790400831

In [7]:
# checking duplicated rows
df_sini.duplicated().sum()

0

## Loading (BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [8]:
# path to dataset
path_df_ret_aseg_cif= "../../../data/interim/5. merged/VIGABT_POLIZAS+BTT_ASEGURADOS+tb_cif.feather"

if not(os.path.exists(path_df_ret_aseg_cif) and os.path.isfile(path_df_ret_aseg_cif)):
    raise Exception("File {} doesn't exists.".format(path_df_ret_aseg_cif))

In [9]:
df_ret_aseg_cif = pd.read_feather(path_df_ret_aseg_cif)
df_ret_aseg_cif.info()

/home/voolkia/anaconda3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729756 entries, 0 to 729755
Data columns (total 39 columns):
CANT_RENOVACION                 709911 non-null float64
CAPITAL_ACCESORIOS              729756 non-null object
CAPITAL_ASEGURADO_COTIZACION    725415 non-null object
CAPITAL_VEHICULO                729756 non-null object
COD_COBRO                       729756 non-null object
COD_COBRO_ANTERIOR              604173 non-null object
COD_POSTAL                      729756 non-null int64
COD_PROD                        729756 non-null int64
COD_RAMO                        729756 non-null int64
COD_RIES                        729756 non-null int64
COD_ZONA_CASCO                  729754 non-null float64
COD_ZONA_RC                     729754 non-null float64
COD_ZONA_ROBO                   729754 non-null float64
CONV_COMISIONARIO               729756 non-null object
FECHA_PROCESO                   729756 non-null datetime64[ns]
FECHA_VENC_POL                  729756 non-null datetime

In [10]:
# checking ID is unique
df_ret_aseg_cif["NUM_SECU_POL"].nunique() / len(df_ret_aseg_cif)

1.0

## Comparing DSS_SINIESTROS_AUTOS vs (BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)
Checking NUM_SECU_POL intersection

In [13]:
# Checking intersection between both datasets
id_common = set(df_sini["NUM_SECU_POL"].unique()).intersection(set(df_ret_aseg_cif["NUM_SECU_POL"].unique()))
len(id_common), len(id_common) / df_sini["NUM_SECU_POL"].nunique() *100, len(id_common) /df_ret_aseg_cif["NUM_SECU_POL"].nunique()*100

(729756, 99.4296552043622, 100.0)

In [14]:
not_in_merged = set(df_sini["NUM_SECU_POL"].unique()) - set(df_ret_aseg_cif["NUM_SECU_POL"].unique())
len(not_in_merged)

4186

**4186 NUM_SECU_POL OUT**

In [21]:
# distrib of missing by year
df_sini[df_sini["NUM_SECU_POL"].isin(not_in_merged)]["FECHA_SINI"].dt.year.value_counts().sort_index()

2017    3040
2018    3145
2019     849
Name: FECHA_SINI, dtype: int64

## Merging DSS_SINIESTROS_AUTOS vs (BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [22]:
# Merging dropping not founded in DSS_SINIESTROS_AUTOS
df_merged = pd.merge(df_sini, df_ret_aseg_cif, on="NUM_SECU_POL", how="left", suffixes=("_sini", "_vigabt_aseg_cif"))
print(len(df_sini), len(df_ret_aseg_cif), len(df_merged))

1144995 729756 1144995


In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144995 entries, 0 to 1144994
Data columns (total 82 columns):
CATASTROFICO                    1144989 non-null object
CERRADURA_BAUL                  504402 non-null object
CERRADURA_DERECHA               504402 non-null object
CERRADURA_IZQUIERDA             504402 non-null object
CODIGO_BAJA                     1562 non-null float64
CODIGO_CARATULA                 30240 non-null float64
CODIGO_REAPERTURA               58512 non-null float64
CODIGO_VEHICULO                 1144905 non-null float64
COD_ACT_BENEF                   463893 non-null float64
COD_CAUSA_SINI                  1144995 non-null float64
COD_POST_OCURRENCIA             1144991 non-null float64
COD_POST_POLIZA                 1144971 non-null float64
COD_POST_TERC                   194056 non-null float64
COD_RAMO_sini                   1144995 non-null int64
COD_RIES_sini                   1144995 non-null float64
CONDICION_ROBO_EXP50            13085 non-null flo

### saving merged dataset

In [24]:
path_to_save = "../../../data/interim/5. merged"
check_filename = "DSS_SINIESTROS_AUTOS+VIGABT_POLIZAS+BTT_ASEGURADOS+tb_cif.feather"
df_merged.to_feather(os.path.join(path_to_save, check_filename))